## JSONファイルからRAG用のベクトルDBを作る

#### task
1. 下記リンクコピペ [ok]
2. 正常動作するように変更 [x]
3. サーバのリソースを占有しないためにLLM部分は適宜unsloth/gpt-oss-20bに変更する [x]

LangChainを使ってシンプルなRAG作ってみた
> https://qiita.com/yagiyanagi/items/bfff61dbd077eb391812#langchain%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%9Frag%E3%81%AE%E6%A7%8B%E7%AF%89%E6%89%8B%E9%A0%86

### install

In [1]:
# !yay -S python-langchain python-langchain-community python-jq python-sentence-transformers
!pip install langchain langchain-community jq sentence-transformers

error: externally-managed-environment

× This environment is externally managed
╰─> To install Python packages system-wide, try 'pacman -S
    python-xyz', where xyz is the package you are trying to
    install.
    
    If you wish to install a non-Arch-packaged Python package,
    create a virtual environment using 'python -m venv path/to/venv'.
    Then use path/to/venv/bin/python and path/to/venv/bin/pip.
    
    If you wish to install a non-Arch packaged Python application,
    it may be easiest to use 'pipx install xyz', which will manage a
    virtual environment for you. Make sure you have python-pipx
    installed via pacman.

note: If you believe this is a mistake, please contact your Python installation or OS distribution provider. You can override this, at the risk of breaking your Python installation or OS, by passing --break-system-packages.
hint: See PEP 668 for the detailed specification.


### 埋め込みモデルの準備

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

/tmp/ipykernel_248960/3077754740.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings()
/tmp/ipykernel_248960/3077754740.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = HuggingFaceEmbeddings()
/usr/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_t

### ベクトルDBの設定

In [3]:
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

### データの読み込み

In [4]:
# langchainのJson読み込みモジュールを使う
from langchain.document_loaders import JSONLoader

loader = JSONLoader(
    file_path="./JVN.json",
    jq_schema=".[]",
    text_content=False
)

docs = loader.load()
for doc in docs:
    doc.page_content = doc.page_content.encode('utf-8').decode('unicode_escape')


In [5]:
docs[0]

Document(metadata={'source': '/home/yokogawa/ドキュメント/MySeniorThesis-main/JVN.json', 'seq_num': 1}, page_content='{"title": "escanav の Linux 用 escan anti-virus におけるバッファエラーの脆弱性", "description": "A vulnerability was found in MicroWord eScan Antivirus 7.0.32 on Linux and classified as critical. Affected by this issue is the function strcpy of the component VirusPopUp. The manipulation leads to stack-based buffer overflow. The attack needs to be approached locally. The exploit has been disclosed to the public and may be used. The vendor was contacted early about this disclosure but did not respond in any way.", "technologies": "escanav", "version": "7.0.32"}')

### データの分割

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
all_splits = text_splitter.split_documents(docs)

### データの保存

In [ ]:
_ = vector_store.add_documents(documents=all_splits)

### Stateの定義

In [ ]:
from langchain_core.documents import Document
from typing_extensions import List, TypedDict

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

### Retrieverの構築

In [ ]:
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

### Generatorの構築

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = """

質問: {question}
コンテキスト: {context}
回答:
"""

def generate(state: State):
    docs_context = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_context})
    response = llm.invoke(messages)
    return {"answer": response.content}

### 制御フローの定義

In [2]:
from langgraph.graph import START, StateGraph

graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

ModuleNotFoundError: No module named 'langgraph.graph'

### 使用例

In [3]:
response = graph.invoke({"question": "なおっちさんはどこで働いていますか？"})
print(response["answer"])

NameError: name 'graph' is not defined

In [4]:
print(response['context'])

NameError: name 'response' is not defined